In [10]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from PIL import Image, ImageTk, ImageDraw, ImageFont
from datetime import datetime
import numpy as np


def get_zodiac_sign(day, month):
    if (month == 12 and day >= 22) or (month == 1 and day <= 19):
        return "Steinbock"
    elif (month == 1 and day >= 20) or (month == 2 and day <= 18):
        return "Wassermann"
    elif (month == 2 and day >= 19) or (month == 3 and day <= 20):
        return "Fisch"
    elif (month == 3 and day >= 21) or (month == 4 and day <= 19):
        return "Widder"
    elif (month == 4 and day >= 20) or (month == 5 and day <= 20):
        return "Stier"
    elif (month == 5 and day >= 21) or (month == 6 and day <= 20):
        return "Zwilling"
    elif (month == 6 and day >= 21) or (month == 7 and day <= 22):
        return "Krebs"
    elif (month == 7 and day >= 23) or (month == 8 and day <= 22):
        return "Loewe"
    elif (month == 8 and day >= 23) or (month == 9 and day <= 22):
        return "Jungfrau"
    elif (month == 9 and day >= 23) or (month == 10 and day <= 22):
        return "Waage"
    elif (month == 10 and day >= 23) or (month == 11 and day <= 21):
        return "Skorpion"
    elif (month == 11 and day >= 22) or (month == 12 and day <= 21):
        return "Schütze"

def select_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        try:
            image = Image.open(file_path)
            image.thumbnail((800, 800), Image.Resampling.LANCZOS)
            user_image = ImageTk.PhotoImage(image)
            image_label.config(image=user_image)
            image_label.image = user_image
            image_label.file_path = file_path
        except Exception as e:
            messagebox.showerror("Fehler", f"Bild konnte nicht geladen werden: {e}")

#def select_image():
 #   file_path = filedialog.askopenfilename()
  #  if file_path:
  #      try:
   #         image = Image.open(file_path)
   #         width, height = image.size
   #         if width > height:
   #             ratio = 800 / float(width)
    #            height = int(float(height) * ratio)
    #            image = image.resize((800, height), Image.LANCZOS)
     #       else:
     #           ratio = 800 / float(height)
     #           width = int(float(width) * ratio)
     #           image = image.resize((width, 800), Image.LANCZOS)
     #       user_image = ImageTk.PhotoImage(image)
     #       image_label.config(image=user_image)
     #       image_label.image = user_image
     #       image_label.file_path = file_path
     #   except Exception as e:
     #      messagebox.showerror("Fehler", f"Bild konnte nicht geladen werden: {e}")


def resize_to_800x800(image_array):
   image = Image.fromarray(image_array)
   image = image.resize((800, 800), Image.LANCZOS)
   return np.array(image)




def show_results():
    name = name_entry.get()
    birth_date = birth_date_entry.get()
    image_path = getattr(image_label, 'file_path', None)
    
    try:
        birth_date_obj = datetime.strptime(birth_date, "%d.%m.%Y")
        day, month = birth_date_obj.day, birth_date_obj.month
        zodiac_sign = get_zodiac_sign(day, month)
    except ValueError:
        messagebox.showerror("Fehler", "Ungültiges Geburtsdatum. Bitte im Format TT.MM.JJJJ eingeben.")
        return

    if not name or not birth_date or not image_path:
        messagebox.showerror("Fehler", "Bitte füllen Sie alle Felder aus und wählen Sie ein Bild aus.")
        return
    
    #Erstellen des Ausgabe-Bildes
    user_image = Image.open(image_path)
    user_image.thumbnail((800, 800), Image.Resampling.LANCZOS)
    user_image = user_image.convert("RGBA")  # Sicherstellen, dass das Bild RGBA ist
    user_image_width, user_image_height = user_image.size


    # Laden und Einfügen des Sternzeichenbildes
    zodiac_image_path = f"zodiac_images/{zodiac_sign.lower()}.png"  # Beispielpfad für Sternzeichenbilder
    try:
        zodiac_image = Image.open(zodiac_image_path)
        zodiac_image.thumbnail((200, 200), Image.Resampling.LANCZOS)
        zodiac_image = zodiac_image.convert("RGBA")  # Sicherstellen, dass das Bild RGBA ist
        zodiac_image_width, zodiac_image_height = zodiac_image.size
    except FileNotFoundError:
        messagebox.showerror("Fehler", f"Sternzeichenbild für {zodiac_sign} nicht gefunden.")
        return

    combined_image = Image.new('RGBA', (user_image_width, user_image_height))
    combined_image.paste(user_image, (0, 0))
    combined_image.paste(zodiac_image, (user_image_width - zodiac_image_width, 0), zodiac_image)

    # Hinzufügen von Name und Geburtsdatum
    draw = ImageDraw.Draw(combined_image)
    font = ImageFont.truetype("arial.ttf", 40)  # Sie können eine andere Schriftart verwenden
    text = f"Name: {name}\nGeburtsdatum: {birth_date}\nDu bist: {zodiac_sign}"
    text_position = (10, user_image_height - 120)  # Platzieren des Textes unten links
    draw.text(text_position, text, font=font, fill="white")

   # Speichern des Ausgabe-Bildes
    save_path = filedialog.asksaveasfilename(defaultextension=".png",
                                             filetypes=[("PNG files", "*.png"), ("All files", "*.*")])
    if save_path:
        combined_image.save(save_path)
        messagebox.showinfo("Erfolg", f"Bild erfolgreich gespeichert unter: {save_path}") 
    
    
    
    # Anzeigen des Ausgabe-Bildes
    combined_image.show()

root = tk.Tk()
root.title("Sternzeichen Bestimmung")

name_label = ttk.Label(root, text="Name:")
name_label.grid(column=0, row=0, padx=10, pady=10)
name_entry = ttk.Entry(root)
name_entry.grid(column=1, row=0, padx=10, pady=10)

birth_date_label = ttk.Label(root, text="Geburtsdatum (TT.MM.JJJJ):")
birth_date_label.grid(column=0, row=1, padx=10, pady=10)
birth_date_entry = ttk.Entry(root)
birth_date_entry.grid(column=1, row=1, padx=10, pady=10)

image_label = ttk.Label(root, text="Kein Bild ausgewählt")
image_label.grid(column=0, row=2, columnspan=2, padx=10, pady=10)
image_button = ttk.Button(root, text="Bild auswählen", command=select_image)
image_button.grid(column=0, row=3, columnspan=2, padx=10, pady=10)

submit_button = ttk.Button(root, text="Ergebnisse anzeigen", command=show_results)
submit_button.grid(column=0, row=4, columnspan=2, padx=10, pady=10)

root.mainloop()


KeyboardInterrupt: 

KeyboardInterrupt: 

: 

In [6]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from PIL import Image, ImageTk, ImageDraw, ImageFont
import numpy as np
from datetime import datetime

def get_zodiac_sign(day, month):
    if (month == 12 and day >= 22) or (month == 1 and day <= 19):
        return "Steinbock"
    elif (month == 1 and day >= 20) or (month == 2 and day <= 18):
        return "Wassermann"
    elif (month == 2 and day >= 19) or (month == 3 and day <= 20):
        return "Fisch"
    elif (month == 3 and day >= 21) or (month == 4 and day <= 19):
        return "Widder"
    elif (month == 4 and day >= 20) or (month == 5 and day <= 20):
        return "Stier"
    elif (month == 5 and day >= 21) or (month == 6 and day <= 20):
        return "Zwilling"
    elif (month == 6 and day >= 21) or (month == 7 and day <= 22):
        return "Krebs"
    elif (month == 7 and day >= 23) or (month == 8 and day <= 22):
        return "Loewe"
    elif (month == 8 and day >= 23) or (month == 9 and day <= 22):
        return "Jungfrau"
    elif (month == 9 and day >= 23) or (month == 10 and day <= 22):
        return "Waage"
    elif (month == 10 and day >= 23) or (month == 11 and day <= 21):
        return "Skorpion"
    elif (month == 11 and day >= 22) or (month == 12 and day <= 21):
        return "Schütze"

def select_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        try:
            image = Image.open(file_path)
            image = image.resize((800, 800), Image.LANCZOS)
            user_image = ImageTk.PhotoImage(image)
            image_label.config(image=user_image)
            image_label.image = user_image
            image_label.file_path = file_path
        except Exception as e:
            messagebox.showerror("Fehler", f"Bild konnte nicht geladen werden: {e}")

def resize_to_800x800(image_array):
    image = Image.fromarray(image_array)
    image = image.resize((800, 800), Image.LANCZOS)
    return np.array(image)

def show_results():
    name = name_entry.get()
    birth_date = birth_date_entry.get()
    image_path = getattr(image_label, 'file_path', None)
    
    try:
        birth_date_obj = datetime.strptime(birth_date, "%d.%m.%Y")
        day, month = birth_date_obj.day, birth_date_obj.month
        zodiac_sign = get_zodiac_sign(day, month)
    except ValueError:
        messagebox.showerror("Fehler", "Ungültiges Geburtsdatum. Bitte im Format TT.MM.JJJJ eingeben.")
        return

    if not name or not birth_date or not image_path:
        messagebox.showerror("Fehler", "Bitte füllen Sie alle Felder aus und wählen Sie ein Bild aus.")
        return
    
    # Erstellen des Ausgabe-Bildes
    user_image = Image.open(image_path)
    user_image = user_image.convert("RGBA")
    user_image_array = np.array(user_image)
    user_image_array = resize_to_800x800(user_image_array)
    user_image = Image.fromarray(user_image_array)
    user_image_width, user_image_height = user_image.size

    # Laden und Einfügen des Sternzeichenbildes
    zodiac_image_path = f"zodiac_images/{zodiac_sign.lower()}.png"
    try:
        zodiac_image = Image.open(zodiac_image_path)
        zodiac_image.thumbnail((200, 200), Image.Resampling.LANCZOS)
        zodiac_image = zodiac_image.convert("RGBA")
        zodiac_image_width, zodiac_image_height = zodiac_image.size
    except FileNotFoundError:
        messagebox.showerror("Fehler", f"Sternzeichenbild für {zodiac_sign} nicht gefunden.")
        return

    combined_image = Image.new('RGBA', (user_image_width, user_image_height))
    combined_image.paste(user_image, (0, 0))
    combined_image.paste(zodiac_image, (user_image_width - zodiac_image_width, 0), zodiac_image)

    # Hinzufügen von Name und Geburtsdatum
    draw = ImageDraw.Draw(combined_image)
    font = ImageFont.truetype("arial.ttf", 40)
    text = f"Name: {name}\nGeburtsdatum: {birth_date}\nDu bist: {zodiac_sign}"
    text_position = (10, user_image_height - 120)
    draw.text(text_position, text, font=font, fill="white")

    # Speichern des Ausgabe-Bildes
    #save_path = filedialog.asksaveasfilename(defaultextension=".png",
    #                                         filetypes=[("PNG files", "*.png"), ("All files", "*.*")])
    #if save_path:
    #    combined_image.save(save_path)
    #    messagebox.showinfo("Erfolg", f"Bild erfolgreich gespeichert unter: {save_path}")

root = tk.Tk()
root.title("Sternzeichen Bestimmung")

name_label = ttk.Label(root, text="Name:")
name_label.grid(column=0, row=0, padx=10, pady=10)
name_entry = ttk.Entry(root)
name_entry.grid(column=1, row=0, padx=10, pady=10)

birth_date_label = ttk.Label(root, text="Geburtsdatum (TT.MM.JJJJ):")
birth_date_label.grid(column=0, row=1, padx=10, pady=10)
birth_date_entry = ttk.Entry(root)
birth_date_entry.grid(column=1, row=1, padx=10, pady=10)

image_label = ttk.Label(root, text="Kein Bild ausgewählt")
image_label.grid(column=0, row=2, columnspan=2, padx=10, pady=10)
image_button = ttk.Button(root, text="Bild auswählen", command=select_image)
image_button.grid(column=0, row=3, columnspan=2, padx=10, pady=10)

submit_button = ttk.Button(root, text="Ergebnisse anzeigen", command=show_results)
submit_button.grid(column=0, row=4, columnspan=2, padx=10, pady=10)

root.mainloop()
